### **Dropout**

* 과적합의 이해 - 학습 결과가 학습 데이터에는 매우 잘 맞지만, 학습 데이터에만 꼭 맞춰져 그 외의 데이터에는 잘 안 맞는다.
* 학습시 전체 신경망 중 **일부**만을 사용하도록 하는 것.
* 즉 학습 단계마다 일부 뉴런을 제거(사용X)함으로써 일부 특징이 특정 뉴런에 고정되는 것을 막아 가중치의 균형을 잡도록 한다.
* 학습시 일부 뉴런을 학습시키지 않기 때문에 신경망이 **충분히 학습되기까지**의 시간은 조금 더 **오래** 걸리는 편이다.

In [0]:
import os, warnings
# 경고 메시지 무시하거나 숨길때(ignore), 다시보이게(default)
# warnings.filterwarnings(action='default')
warnings.filterwarnings(action='ignore')

In [0]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data', one_hot=True)
type(mnist)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [0]:
## 신경망 모델 구성하기
## 1. 문제와 답
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

## **미니배치의 이해**

* 이미지를 하나하나 학습시키는 것보다 여러 개를 한꺼번에 학습시키는 쪽이 효과가 좋다.
* 많은 메모리와 높은 컴퓨터 성능이 필요하므로 일반적으로 데이터를 적당한 크기로 잘라 학습시킨다.
  * 미니배치라고 한다.
* tf.float32, [None, 784] => None의 자리에는 한번에 학습시킬 이미지의 개수를 지정하는 값이 들어감. 즉, 배치 크기를 지정하는 자리

In [0]:
# 미니 배치
batch_size = 100

In [0]:
## 신경망
# 입력층 784
# 첫번째 256
# 두번째 256
# 출력층 10

### tf.nn.dropout(Layer, 비율)

In [0]:
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X, W1))
L1 = tf.nn.dropout(L1, keep_prob)

W2 = tf.Variable(tf.random_normal([256,256],stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1, W2))
L2 = tf.nn.dropout(L2, keep_prob)  # 뒤의 뉴런을 0.8만 이용

W3 = tf.Variable(tf.random_normal([256,10],stddev=0.01))
model = tf.matmul(L2, W3)

print(W3)
print(model)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
<tf.Variable 'Variable_2:0' shape=(256, 10) dtype=float32_ref>
Tensor("MatMul_2:0", shape=(?, 10), dtype=float32)


In [0]:
### 비용함수, 최적화함수
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,
                                                        labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [0]:
### 세션 초기화 훈련
sess= tf.Session()
sess.run(tf.global_variables_initializer())

### **배치 사이즈 지정**

* Mini-batch 크기가 전체 트레이닝 셋 데이터 사이즈인 m과 같다면 이것은 Batch gradient descent
   * 데이터가 별로 없다면 batch gradient descent 사용
* Mini-batch 크기가 1이라면 Stochastic gradient descent
   * **적은 메모리**로 동작 가능
   * 64, 128, 256, 512 사이즈 선택

In [0]:
## batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [0]:
# 데이터 전체 학습 15회

for epoch in range(15) :
  total_cost = 0

  for i in range(total_batch) :
    batch_image, batch_label = mnist.train.next_batch(batch_size)

    _, cost_val = sess.run([optimizer, cost],
                          feed_dict={X: batch_image,
                                    Y: batch_label,
                                    keep_prob : 0.8})

    # 총 손실
    total_cost = total_cost + cost_val

  print("Epoch : %4d" % (epoch +1),
        "평균 cost :","{:.3f}".format(total_cost / total_batch))

Epoch :    1 평균 cost : 0.429
Epoch :    2 평균 cost : 0.169
Epoch :    3 평균 cost : 0.112
Epoch :    4 평균 cost : 0.088
Epoch :    5 평균 cost : 0.070
Epoch :    6 평균 cost : 0.061
Epoch :    7 평균 cost : 0.054
Epoch :    8 평균 cost : 0.048
Epoch :    9 평균 cost : 0.040
Epoch :   10 평균 cost : 0.037
Epoch :   11 평균 cost : 0.033
Epoch :   12 평균 cost : 0.029
Epoch :   13 평균 cost : 0.030
Epoch :   14 평균 cost : 0.027
Epoch :   15 평균 cost : 0.027
